# Synapses

Synapses model the temporal dynamics of neural connections in `brainpy.state`. This document explains
how synapses work, what models are available, and how to use them effectively.

## Overview

Synapses provide temporal filtering of spike trains, transforming discrete spikes into continuous currents or conductances. They model:

- **Postsynaptic potentials** (PSPs)
- **Temporal integration** of spike trains
- **Synaptic dynamics** (rise and decay)

In BrainPy's architecture, synapses are part of the projection system:

```text
Spikes → [Connectivity] → [Synapse] → [Output] → Neurons
                              ↑
                      Temporal filtering
```


## Basic Usage

### Creating Synapses

Synapses are typically created as part of projections:

In [ ]:
import brainstate
import braintools
import brainunit as u
import jax.numpy as jnp
import matplotlib.pyplot as plt

import brainpy

In [11]:
# Create neurons for demonstration
neurons = brainpy.state.LIF(50, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Create synapse descriptor
syn = brainpy.state.Expon(
    in_size=100,  # Number of synapses
    tau=5. * u.ms  # Time constant
)

# Use in projection
projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(100, 50, 0.1, 0.5),
    syn=syn,  # Synapse here
    out=brainpy.state.CUBA(),
    post=neurons
)

### Synapse Lifecycle

1. **Creation**: Define synapse with `()` method
2. **Integration**: Include in projection
3. **Update**: Called automatically by projection
4. **Access**: Read synaptic variables as needed

In [12]:
# Example presynaptic spikes
presynaptic_spikes = jnp.zeros(100)  # 100 presynaptic neurons

projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.AllToAll(100, 100, 0.1, 0.5),
    syn=brainpy.state.Expon(100, tau=5.0),
    out=brainpy.state.COBA(E=0),
    post=neurons,
)

# During simulation
projection(presynaptic_spikes)  # Updates synapse internally

# Access synaptic variable
synaptic_current = projection.syn

## Available Synapse Models

### Expon (Single Exponential)

The simplest and most commonly used synapse model.

**Mathematical Model:**


$$
\tau \frac{dg}{dt} = -g
$$
When spike arrives: 
$g \leftarrow g + 1$

**Impulse Response:**


$$
g(t) = \exp(-t/\tau)
$$
**Example:**

In [13]:
syn = brainpy.state.Expon(
    size=100,
    tau=5. * u.ms,
    g_initializer=braintools.init.Constant(0. * u.mS)
)

**Parameters:**

- `size`: Number of synapses
- `tau`: Decay time constant
- `g_initializer`: Initial synaptic variable (optional)

**Key Features:**

- Single time constant
- Fast computation
- Instantaneous rise

**Use cases:**

- General-purpose modeling
- Fast simulations
- When precise kinetics are not critical

**Behavior:**

In [14]:
# Response to single spike at t=0
# g(t) = exp(-t/τ)
# Fast rise, exponential decay

### Alpha Synapse

A more realistic model with non-instantaneous rise time.

**Mathematical Model:**


$$
\begin{aligned}
\tau \frac{dh}{dt} &= -h
\tau \frac{dg}{dt} &= -g + h
\end{aligned}
$$
When spike arrives: $ h\leftarrow h + 1 $

**Impulse Response:**


$$
g(t) = \frac{t}{\tau}\exp(-t/\tau)
$$
**Example:**

In [15]:
syn = brainpy.state.Alpha(
    size=100,
    tau=5. * u.ms,
    g_initializer=braintools.init.Constant(0. * u.mS)
)

**Parameters:**

Same as Expon, but produces alpha-shaped response.

**Key Features:**

- Smooth rise and fall
- Biologically realistic
- Peak at t = τ

**Use cases:**

- Biological realism
- Detailed cortical modeling
- When kinetics matter

**Behavior:**

In [71]:
# Response to single spike at t=0
# g(t) = (t/τ) * exp(-t/τ)
# Gradual rise to peak at τ, then decay

### AMPA (Excitatory)

Models AMPA receptor dynamics for excitatory synapses.

**Mathematical Model:**

Similar to Alpha, but with parameters tuned for AMPA receptors.

**Example:**

In [16]:
syn = brainpy.state.AMPA(
    size=100,
    tau=2. * u.ms,  # Fast AMPA kinetics
    g_initializer=braintools.init.Constant(0. * u.mS)
)

**Key Features:**

- Fast kinetics (τ ≈ 2 ms)
- Excitatory receptor
- Biologically parameterized

**Use cases:**

- Excitatory synapses
- Cortical pyramidal neurons
- Biological realism

### GABAa (Inhibitory)

Models GABAa receptor dynamics for inhibitory synapses.

**Mathematical Model:**

Similar to Alpha, but with parameters tuned for GABAa receptors.

**Example:**

In [17]:
syn = brainpy.state.GABAa(
    size=100,
    tau=10. * u.ms,  # Slower GABAa kinetics
    g_initializer=braintools.init.Constant(0. * u.mS)
)

**Key Features:**

- Slower kinetics (τ ≈ 10 ms)
- Inhibitory receptor
- Biologically parameterized

**Use cases:**

- Inhibitory synapses
- GABAergic interneurons
- Biological realism

## Synaptic Variables

### The Descriptor Pattern

BrainPy synapses use a descriptor pattern:

In [18]:
# Create descriptor (not yet instantiated)
syn_desc = brainpy.state.Expon(in_size=100, tau=5 * u.ms)

# Define neurons for example
example_neurons = brainpy.state.LIF(100, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Instantiated within projection
example_projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(100, 100, 0.1, 0.5),
    syn=syn_desc,
    out=brainpy.state.CUBA(),
    post=example_neurons
)

# Access instantiated synapse
actual_synapse = example_projection.syn
g_value = actual_synapse

### Why Descriptors?

- **Deferred instantiation**: Created when needed
- **Reusability**: Same descriptor for multiple projections
- **Flexibility**: Configure before instantiation

### Accessing Synaptic State

In [19]:
# Define neurons for this example
demo_neurons = brainpy.state.LIF(100, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Within projection
demo_projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(100, 100, conn_num=10, conn_weight=0.5),
    syn=brainpy.state.Expon(in_size=100, tau=5 * u.ms),
    out=brainpy.state.CUBA(),
    post=demo_neurons
)

# Initialize states
brainstate.nn.init_all_states(demo_projection)

# Access the synaptic conductance state
synaptic_var = demo_projection.syn.g.value  # Current value with units

# Convert to array for plotting
g_array = u.get_magnitude(synaptic_var)

## Synaptic Dynamics Visualization

### Comparing Different Models

In [ ]:
# Set simulation timestep
brainstate.environ.set(dt=0.1 * u.ms)

# Create different synapses (without unit initializers to avoid mismatch)
expon = brainpy.state.Expon(100, tau=5 * u.ms)
alpha = brainpy.state.Alpha(100, tau=5 * u.ms)
ampa = brainpy.state.AMPA(100, T=2 * u.ms)
gaba = brainpy.state.GABAa(100, T=10 * u.ms)

# Initialize
for syn in [expon, alpha, ampa, gaba]:
    brainstate.nn.init_all_states(syn)

# Single spike at t=0 (dimensionless spike count)
spike_input = jnp.zeros(100)
spike_input = spike_input.at[0].set(1.0)

# Simulate
times = u.math.arange(0 * u.ms, 50 * u.ms, 0.1 * u.ms)
responses = {
    'Expon': [],
    'Alpha': [],
    'AMPA': [],
    'GABAa': []
}

for syn, name in zip(
    [expon, alpha, ampa, gaba],
    ['Expon', 'Alpha', 'AMPA', 'GABAa']
):
    # Re-initialize for clean start
    brainstate.nn.init_all_states(syn)
    for i, t in enumerate(times):
        with brainstate.environ.context(t=t, i=i):
            if i == 0:
                syn(spike_input)
            else:
                syn(jnp.zeros(100))
            # Get the value (may have units or not)
            g_val = syn.g.value[0]
            if hasattr(g_val, 'magnitude'):
                responses[name].append(float(u.get_magnitude(g_val)))
            else:
                responses[name].append(float(g_val))

# Plot
plt.figure(figsize=(10, 6))
for name, response in responses.items():
    response_array = jnp.array(response)
    plt.plot(u.get_magnitude(times),
             response_array,
             label=name, linewidth=2)

plt.xlabel('Time (ms)')
plt.ylabel('Synaptic Variable (normalized)')
plt.title('Comparison of Synapse Models (Single Spike)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Integration with Projections

### Complete Example

In [ ]:
import brainpy
import brainstate
import brainunit as u

# Set simulation timestep
brainstate.environ.set(dt=0.1 * u.ms)

# Create neurons
pre_neurons = brainpy.state.LIF(80, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)
post_neurons = brainpy.state.LIF(100, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Create projection with exponential synapse
projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(
        80, 100, 0.1, 0.5
    ),
    syn=brainpy.state.Expon(100, tau=5 * u.ms),
    out=brainpy.state.CUBA(),
    post=post_neurons
)

# Initialize all states
brainstate.nn.init_all_states([pre_neurons, post_neurons, projection])


# Simulation function
def update(t, i, input_current):
    with brainstate.environ.context(t=t, i=i):
        # Update presynaptic neurons
        pre_neurons(input_current)

        # Get spikes and propagate through projection
        spikes = pre_neurons.get_spike()
        projection(spikes)

        # Update postsynaptic neurons
        post_neurons(0 * u.nA)

        return post_neurons.get_spike()


# Run simulation
times = u.math.arange(0 * u.ms, 100 * u.ms, 0.1 * u.ms)
indices = u.math.arange(times.size)

# Create input current for all timesteps
input_currents = jnp.ones(80) * 2 * u.nA

results = brainstate.transform.for_loop(
    lambda t, i: update(t, i, input_currents),
    times,
    indices
)

## Short-Term Plasticity

Synapses can be combined with short-term plasticity (STP):

Short-term plasticity (STP) can be combined with synapses
Note: STP is typically implemented as part of custom synapse models
or as a separate plasticity mechanism in the projection

Example conceptual usage (not directly supported in current API):
The idea is to modulate synaptic efficacy based on recent activity
- Facilitation: increases with repeated activation
- Depression: decreases with repeated activation

For implementing STP, you would typically:
1. Create a custom synapse class that includes STP dynamics
2. Or use a separate plasticity module that modulates connection weights

See the plasticity documentation for more details on implementing
short-term plasticity mechanisms

See plasticity for more details on STP.

## Custom Synapses

### Creating Custom Synapse Models

You can create custom synapse models by inheriting from `Synapse`:

In [ ]:
import brainstate
import braintools
from brainpy.state import AlignPost


class MyCustomSynapse(AlignPost):
    """Custom synapse with double exponential dynamics."""

    def __init__(self, size, tau1, tau2, g_initializer=None, **kwargs):
        super().__init__(size, **kwargs)

        self.tau1 = tau1
        self.tau2 = tau2

        # Synaptic variable - initialize without units for compatibility
        if g_initializer is None:
            g_initializer = braintools.init.Constant(0.)
        self.g = brainstate.ShortTermState(g_initializer(size))

    def update(self, x):
        # Get time step
        dt = brainstate.environ.get_dt()

        # Custom dynamics: simple decay
        # dg/dt = -g/tau1
        decay = -self.g.value / self.tau1
        self.g.value = self.g.value + decay * dt

        # Add spike input if provided
        if x is not None:
            self.g.value = self.g.value + x / self.tau2

        return self.g.value

Usage:

In [ ]:
# Define neurons for custom synapse example
custom_neurons = brainpy.state.LIF(100, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Create custom synapse descriptor using () method
syn_desc = MyCustomSynapse(
    size=100,
    tau1=5 * u.ms,
    tau2=10 * u.ms
)

# Use in projection
custom_projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(100, 100, 0.1, 0.5),
    syn=syn_desc,
    out=brainpy.state.CUBA(),
    post=custom_neurons
)

print("Custom synapse projection created successfully!")

## Choosing the Right Synapse

### Decision Guide


   * - Model
     - When to Use
     - Pros
     - Cons
   * - Expon
     - General purpose, speed
     - Fast, simple
     - Unrealistic rise
   * - Alpha
     - Biological realism
     - Realistic kinetics
     - Slower computation
   * - AMPA
     - Excitatory, fast
     - Biologically accurate
     - Specific use case
   * - GABAa
     - Inhibitory, slow
     - Biologically accurate
     - Specific use case

### Recommendations

**For machine learning / SNNs:**
   Use `Expon` for speed and simplicity.

**For biological modeling:**
   Use `Alpha`, `AMPA`, or `GABAa` for realism.

**For cortical networks:**
   - Excitatory: `AMPA` (τ ≈ 2 ms)
   - Inhibitory: `GABAa` (τ ≈ 10 ms)

**For custom dynamics:**
   Implement custom synapse class.

## Performance Considerations

### Computational Cost


   * - Model
     - Relative Cost
     - Notes
   * - Expon
     - 1x (baseline)
     - Single state variable
   * - Alpha
     - 2x
     - Two state variables
   * - AMPA/GABAa
     - 2x
     - Similar to Alpha

### Optimization Tips

1. **Use Expon when possible**: Fastest option

2. **Batch operations**: Multiple synapses together

In [ ]:
# Define neurons for batch optimization example
batch_neurons = brainpy.state.LIF(1000, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Good: Single projection with 1000 synapses
good_proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(1000, 1000, 0.1, 0.5),
    syn=brainpy.state.Expon(1000, tau=5 * u.ms),
    out=brainpy.state.CUBA(),
    post=batch_neurons
)

3. **JIT compilation**: Always use for simulations

In [84]:
# Define neurons and projection for JIT example
jit_neurons = brainpy.state.LIF(100, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)
jit_projection = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(100, 100, 0.1, 0.5),
    syn=brainpy.state.Expon(100, tau=5 * u.ms),
    out=brainpy.state.CUBA(),
    post=jit_neurons
)

# Initialize
brainstate.nn.init_all_states([jit_neurons, jit_projection])

# Example spikes
spikes = jnp.zeros(100)


@brainstate.transform.jit
def step():
    jit_projection(spikes)
    jit_neurons(0 * u.nA)

## Common Patterns

### Excitatory-Inhibitory Balance

In [85]:
# Define neurons and sizes for E-I balance example
post_size = 100
ei_neurons = brainpy.state.LIF(post_size, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Excitatory projection (fast)
E_proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(80, post_size, 0.1, 0.5),
    syn=brainpy.state.Expon(post_size, tau=2 * u.ms),
    out=brainpy.state.CUBA(),
    post=ei_neurons
)

# Inhibitory projection (slow)
I_proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(20, post_size, 0.1, 0.5),
    syn=brainpy.state.Expon(post_size, tau=10 * u.ms),
    out=brainpy.state.CUBA(),
    post=ei_neurons
)

### Multiple Receptor Types

In [ ]:
# Define neurons and size for multiple receptor example
receptor_size = 100
receptor_neurons = brainpy.state.LIF(receptor_size, V_rest=-65 * u.mV, V_th=-50 * u.mV, tau=10 * u.ms)

# Fast excitatory (AMPA-like) - using Expon with fast time constant
ampa_proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(80, receptor_size, 0.1, 0.3),
    syn=brainpy.state.Expon(receptor_size, tau=2 * u.ms),  # Fast AMPA kinetics
    out=brainpy.state.COBA(E=0 * u.mV),
    post=receptor_neurons
)

# Slow excitatory (NMDA-like) - using Expon with slow time constant
nmda_proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(80, receptor_size, 0.1, 0.3),
    syn=brainpy.state.Expon(receptor_size, tau=100 * u.ms),  # Slow NMDA kinetics
    out=brainpy.state.COBA(E=0 * u.mV),
    post=receptor_neurons
)

# Fast inhibitory (GABAa-like) - using Expon with medium time constant
gaba_proj = brainpy.state.AlignPostProj(
    comm=brainstate.nn.EventFixedProb(20, receptor_size, 0.1, 0.5),
    syn=brainpy.state.Expon(receptor_size, tau=10 * u.ms),  # GABAa kinetics
    out=brainpy.state.COBA(E=-80 * u.mV),
    post=receptor_neurons
)

print("Multiple receptor type projections created successfully!")

## Summary

Synapses in `brainpy.state`:

✅ **Multiple models**: Expon, Alpha, AMPA, GABAa

✅ **Temporal filtering**: Convert spikes to continuous signals

✅ **Descriptor pattern**: Flexible, reusable configuration

✅ **Integration ready**: Seamless use in projections

✅ **Extensible**: Easy custom synapse models

✅ **Physical units**: Proper unit handling throughout

## Next Steps

- Learn about [projections](projections) for complete connectivity
- Explore [plasticity](state-management) for learning rules
- Follow [tutorials](../tutorials/index) for practice
- See [examples](../examples/gallery) for network examples